This notebook uses [GPTNeo](https://github.com/EleutherAI/GPTNeo) by [EleutherAI](eleuther.ai) to fine tune the model and predict multiple files.

To fine-tune the model copy the excel file containg the dataset to "train/data" folder on the drive

In [ ]:
import pandas as pd
from pandas import read_excel
from pathlib import Path
import numpy as np
import lxml.html
import string
import os
import re

In [ ]:
def read_csv(path):
    #file_name = '02_batch_import_Dior.xlsx'
    #df = read_excel(Path(path,file_name), sheet_name = my_sheet,keep_default_na=False)
    df = pd.read_excel(path, sheet_name='Sheet1')
    df.reset_index(drop=True, inplace=True)
    return clean(df)

def clean(df):
    to_keep=["brand","category","description","color","gender","pattern","neckline","sleeves","material"]
    to_drop=[]
    for col in df.columns:
        if col not in to_keep:
            to_drop.append(col)
    df.drop(to_drop, inplace=True, axis=1)
    return df

In [ ]:
def write_as_txt(df,write_dir):
   # Shuffle indices and split the data Train 70%, val 30%
   df = df.iloc[np.random.permutation(len(df))]
   #train, validate= np.split(df.sample(frac=1), [int(.7*len(df))])
   train = df
   write(train, "train",write_dir)
   #write(validate,"validate",write_dir)
   #write(test,"test",write_dir)

def write(df, t,write_dir):
    dir = write_dir
    if t=="train":
        path = dir+"train_text/"
    elif t=="validate":
        path = dir+"val_text/"
    #elif t=="test":
    #    path = dir+"test_text/"
    #ref_path = dir+"test_text_ref/"
    c=0
    data= df.to_dict('index')
    #if t == "test":
    #    #write the test set with and without the lables to have a reference
    #    for k,value in data.items():
    #        value = {k:v for k,v in value.items() if str(v)!= '' and str(v).strip() != '' and str(v)!='nan' and str(v)!='null'}
    #        write_dict(value,ref_path+"product"+str(c)+".txt","n")
    #        c+=1
    #    c=0
    #    for k,value in data.items():
    #        value = {k:v for k,v in value.items() if str(v)!= '' and str(v).strip() != '' and str(v)!='nan' and str(v)!='null'}
    #        write_dict(value,path+"product"+str(c)+".txt","t")
    #        c+=1 
    #else :
    for k,value in data.items():
        value = {k:v for k,v in value.items() if str(v)!= '' and str(v).strip() != '' and str(v)!='nan' and str(v)!='null' and str(v)!=  '[]'}
        write_dict(value,path+"product"+str(c)+".txt","n")
        c+=1
    print("writing "+t+ " successful")

# writes the file with format:
# when type in "n" for normal
# {"tag1" : "value1", "tag2": "value2", ....} \n description: "description_en" \n ### \n
# when type is "t" for test
# {"tag1" : "value1", "tag2": "value2", ....} \n description: 
def write_dict(dict, path, type):
    feat,desc = get_data(dict) # only write the features and the generated descriptions
    if (desc):
        with open(path, 'w') as f:
            txt = feat + "\n"
            if type == "n":
                if txt[-1] != '\n':
                    txt+='\n'
                txt += "description: " + desc + "\n###\n"
            txt = clean_txt(txt)
            print(txt,file =f)
            
def clean_txt(st):
    st = st.strip()
    st = st.replace('"','')
    st = st.replace('[','')
    st = st.replace(']','')
    st = st.replace("'",'')
    return st

def get_data(dict):
    feat = dict
    desc = dict.pop("description", None)
    feat = str(feat).replace('{','').replace('}','').replace("'",'')
    return feat,desc

In [ ]:
read_dir = "/content/drive/MyDrive/train/raw/"
write_dir = "/content/drive/MyDrive/train/concat/"
write_as_txt(read_csv(read_dir),write_dir)

In [ ]:
!cat /content/drive/MyDrive/train/train_text/*.txt -> /content/drive/MyDrive/train/concat.txt

In [ ]:
import os
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/gpt-neo
%cd gpt-neo
!pip3 install -q -r requirements.txt
pretrained_model = None
dataset = None


Cloning into 'gpt-neo'...
remote: Enumerating objects: 3904, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 3904 (delta 168), reused 183 (delta 98), pack-reused 3609
Receiving objects: 100% (3904/3904), 1.49 MiB | 8.85 MiB/s, done.
Resolving deltas: 100% (2258/2258), done.
/content/gpt-neo
     |████████████████████████████████| 361 kB 5.5 MB/s 
     |████████████████████████████████| 14.5 MB 92 kB/s 
     |████████████████████████████████| 106 kB 57.3 MB/s 
     |████████████████████████████████| 454.4 MB 9.0 kB/s 
     |████████████████████████████████| 3.4 MB 56.9 MB/s 
     |████████████████████████████████| 2.9 MB 45.1 MB/s 
     |████████████████████████████████| 1.5 MB 42.8 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 462 kB 63.9 MB/s 
     |████████████████████████████████| 4.0 MB 37.3 MB/s 
     |████████████████████████████████| 1.2 MB 72.2 MB/s 
  

In [ ]:
!pip install -U tensorflow-gcs-config==2.1.3
!pip install -q t5 tensorflow-text==2.3

     |████████████████████████████████| 98 kB 3.9 MB/s 
  Attempting uninstall: tensorflow-gcs-config
    Found existing installation: tensorflow-gcs-config 2.6.0
    Uninstalling tensorflow-gcs-config-2.6.0:
      Successfully uninstalled tensorflow-gcs-config-2.6.0
     |████████████████████████████████| 152 kB 5.4 MB/s 
     |████████████████████████████████| 2.6 MB 36.0 MB/s 
     |████████████████████████████████| 320.6 MB 40 kB/s 
     |████████████████████████████████| 2.9 MB 42.3 MB/s 
     |████████████████████████████████| 459 kB 52.6 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 269 kB 45.8 MB/s 
     |████████████████████████████████| 4.0 MB 35.5 MB/s 
     |████████████████████████████████| 90 kB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 49.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following de

Choose the following options:
1. re-initialize this configuration [1]
2. the google account with the cloud storage [1]
3. gpt project [10]
4. No [n]

In [ ]:
from google.colab import auth
auth.authenticate_user()
#!gcloud auth login
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: zeroundici@gmail.com
  project: gpt-j-325212

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] zeroundici@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [zeroundici@gmail.com].

Pick cloud project 

In [ ]:
path_to_cloud_bucket = 'gs://test-gpt-j/' 

Mount the drive where the generated predictions will be stored.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset

In [ ]:
# Select a Dataset:
import os
dataset_path = "/content/drive/MyDrive/train/concat/"
dataset_name = 'dataset_name'
out_name = dataset_name + "_tokenized"

In [ ]:
# Tokenize Data
!python data/create_tfrecords.py --input_dir $dataset_path --name $dataset_name --files_per 1000 --output_dir $out_name --write_dataset_config --processes 1

# copy the data to your bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'
copy_loc = path_to_cloud_bucket + "dataset/"
!gsutil -m cp -r /content/gpt-neo/$out_name $copy_loc
!gsutil ls $path_to_cloud_bucket

2021-10-06 12:54:57.650397: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 1.04M/1.04M [00:00<00:00, 6.80MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.86MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 8.30MB/s]
Writing TFRecord Files to flipkart_tokenized/. Parsed 0 input files. files_written : 0it [00:02, ?it/s]
{'discarded': 0, 'processed': 1, 'successful': 1}
Copying file:///content/gpt-neo/flipkart_tokenized/flipkart_0_134.tfrecords [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/526.0 KiB.                                    
gs://test-gpt-j/GPT3_XL/
gs://test-gpt-j/datasets/
gs://test-gpt-j/eval_datasets/
gs://test-gpt-j/flipkart_dataset/
gs://test-gpt-j/telablu_dataset/


# Configs
dataset configs
If dataset_name was changed, change the name of the file being written accordingly.

In [ ]:
%%writefile configs/dataset_configs/dataset_name.json

{
  "path": "gs://test-gpt-j/dataset/dataset_name_*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}


Writing configs/dataset_configs/flipkart.json


Model configs

If dataset_name was changed, change the value for "dataset" in the config accordingly.

In [ ]:
%%writefile configs/GPT3_XL.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 256,
    "attn_dropout": 0,
    "train_steps": 600000,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 4,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["dataset_name", null, null, null]],
    "model": "GPT",
    "model_path": "gs://test-gpt-j/",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:4,y:2",
    "layout": "intermediate_expanded:x,heads:x,vocab:n_vocab,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16"
}

Writing configs/GPT3_XL.json


#Pretrained Model

In [ ]:
pretrained_model = 'GPT3_XL' 
path_to_local_weights = f"/content/gpt-neo/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"

In [ ]:
pretrained_model = 'GPT3_XL' 
!wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/$pretrained_model/"
path_to_local_weights = f"/content/gpt-neo/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"


--2021-10-06 12:56:52--  https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp’

the-eye.eu/public/A     [ <=>                ]  10.30K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2021-10-06 12:56:53 (72.6 MB/s) - ‘the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp’ saved [10552]

Loading robots.txt; please ignore errors.
--2021-10-06 12:56:55--  https://the-eye.eu/robots.txt
Reusing existing connection to the-eye.eu:443.
HTTP request sent, awaiting response... 200 OK
Length: 4 [text/plain]
Saving to: ‘the-eye.eu/robots.txt’

the-eye.eu/robots.t 100%[===================>]       4  --.-KB/s    in 0s      

2021-10-06 12:56:55 (673 KB/s) - ‘the-eye.eu/robots.txt’ saved [4/

In [ ]:

bucket_base = "gs://" + path_to_cloud_bucket.replace('gs://', '').split('/')[0]
!gsutil -m cp -r $path_to_local_weights $bucket_base

If dataset_name was modified, change teh value of "dataset" in "mods" accordingly.

In [ ]:
import json
from pprint import pprint

path_to_model = "" 
batch_size = 8 
dset = "prod_desc_gpt_j"  
mesh_shape = "x:4,y:2"
train_steps = 1000 
steps_per_checkpoint = 500 
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None and dset == "":
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": [["dataset_name", None, None, None]],
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

MODEL PATH: gs://test-gpt-j/GPT3_XL

{'activation_function': 'gelu',
 'attention_types': [[['global', 'local'], 12]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['pile', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 10,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'batch:x,memory_length:y,embd:y',
 'lr': 0.0002,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:128,y:2',
 'model_path': 'gs://neo-d/models/GPT3_XL_Pile',
 'n_ctx': 2048,
 'n_embd': 2048,
 'n_head': 16,
 'n_layer': 24,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'precision': 'bfloat16',
 'predict_batch_size': 128,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 512,
 'train_steps': 400000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'activation_function': '

### Begin Fine-Tuning


Give permissions to the service by adding it as a storage admin of the bucket if permission denied errors rise.

In [ ]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --tpu colab

2021-10-06 13:01:58.960052: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term
Current step 362000
Saving config to gs://test-gpt-j/GPT3_XL
2021-10-06 13:02:07.946957: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-06 13:02:07.953572: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2199995000 Hz
2021-10-06 13:02:07.953835: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eded2bbd40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-06 13:02:07.953882: I tensorflow/compiler/xla/serv

#Evaluate the model

If you want to evaluate the model, you have to make another concat.txt file, tokenize it and upload to the bucket as before and run the model on eval mode.

In [ ]:
# Select a Dataset:
import os
dataset_path = "/content/drive/MyDrive/eval_dataset/"
dataset_name = 'eval_dataset'
out_name = dataset_name + "_tokenized"

In [ ]:
# Tokenize Data
!python data/create_tfrecords.py --input_dir $dataset_path --name $dataset_name --files_per 1000 --output_dir $out_name --write_dataset_config --processes 1

# copy the data to your bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'
copy_loc = path_to_cloud_bucket + "eval_datasets/"
!gsutil -m cp -r /content/gpt-neo/$out_name $copy_loc
!gsutil ls $path_to_cloud_bucket

2021-10-06 13:59:04.518580: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Writing TFRecord Files to eval_dataset_tokenized/. Parsed 0 input files. files_written : 0it [00:01, ?it/s]
{'discarded': 0, 'processed': 1, 'successful': 1}
Copying file:///content/gpt-neo/eval_dataset_tokenized/eval_dataset_0_134.tfrecords [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/526.0 KiB.                                    
gs://test-gpt-j/GPT3_XL/
gs://test-gpt-j/datasets/
gs://test-gpt-j/eval_datasets/
gs://test-gpt-j/flipkart_dataset/
gs://test-gpt-j/telablu_dataset/


create dataset configs

In [ ]:
%%writefile configs/dataset_configs/eval_dataset.json

{
  "path": "",
  "eval_path": "gs://test-gpt-j/eval_datasets/*.tfrecords",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}


Writing configs/dataset_configs/eval_dataset.json


update the configs to point to the dataset

In [ ]:
import json
from pprint import pprint

batch_size = 8
assert pretrained_model is not None
with open(f'configs/{pretrained_model}.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [["eval_dataset", None, None, None]]
  mods = {
          "datasets": dset_val,
          "eval_steps": 139 // batch_size,
          "train_batch_size": batch_size,
          "eval_batch_size": batch_size,
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

{'activation_function': 'gelu',
 'attention_types': [[['global', 'local'], 12]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['prod_desc_gpt_j', 'flipkart', None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 0,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'intermediate_expanded:x,heads:x,memory_length:y,embd:y',
 'lr': 0.0002,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:4,y:2',
 'model_path': 'gs://test-gpt-j/GPT3_XL',
 'n_ctx': 2048,
 'n_embd': 2048,
 'n_head': 16,
 'n_layer': 24,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'precision': 'bfloat16',
 'predict_batch_size': 8,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 8,
 'train_steps': 363000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'activation_function': 'gelu',
 'a

Run evaluation

In [ ]:
!python3 main.py --eval --tpu colab --model $pretrained_model

2021-10-06 13:59:56.508680: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term
Current step 363000
Saving config to gs://test-gpt-j/GPT3_XL
2021-10-06 14:00:03.930677: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-06 14:00:03.936956: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2199995000 Hz
2021-10-06 14:00:03.937216: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563f49e8db80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-06 14:00:03.937261: I tensorflow/compiler/xla/serv